作为例子，我们将计算员工foo和bar的净工资：

In [20]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('X,Y,Z, salary, tax_rate, tax_rate_for_salary_above, net_salary')
salary['foo'] = 60
salary['bar'] = 110

# Python equivalent【只是作为展示， 实际上_salary 并没有被 define】
# _salary = dict()
# _salary['foo'] = 60
# _salary['bar'] = 110

# give me all the X and Y so that the salary of X is Y
print(salary[X]==Y)
print({X.data[i]:Y.data[i] for i in range(len(X.data))})        #【真正转化为字典的写法】
# python equivalent
# print(_salary.items())

X   | Y  
----|----
foo | 60 
bar | 110
{'foo': 60, 'bar': 110}


请注意，逻辑函数名称（例如 salary），以小写字母开头。 函数为给定参数定义一个值。它类似于Python字典。

一个函数可以用值查询。 一个函数对同一个参数只能有一个值（后来值会覆盖旧值）：

In [21]:
# foo now has a salary of 70
salary['foo'] = 70
print(salary['foo']==Y)

Y 
--
70


一个函数也可以用键查询：

In [22]:
# give me all the X that have a salary of 110
print(salary[X]==110)
# procedural equivalent in python
# for i, j in _salary.items():
#     if j==110:
#         print i, '-->', j
#  Notice that there is a implicit loop in the query.

X  
---
bar


注意查询中有一个隐式循环（因此这种查询效率比较低）。

查询可以测试一个标准的否定：

In [23]:
# A query can test the negation of a criteria.
print((salary[X]==Y) & ~(Y==110))

X   | Y 
----|---
foo | 70


现在让我们定义一个全球税率。我们将使用 None 函数参数：

In [24]:
# Let's now define a global tax rate. We'll use None for the function argument:
# the standard tax rate is 33%
+(tax_rate[None]==0.33)

# 一个函数可以在公式中调用：
# give me the net salary for all X
print((Z==salary[X]*(1-tax_rate[None])))

X   | Z                
----|------------------
foo | 46.89999999999999
bar | 73.69999999999999


In [25]:
# give me the net salary of foo
print(net_salary['foo']==Y)
print(net_salary[X]<50)

Y                
-----------------
46.89999999999999
X  
---
foo


现在让我们定义一个全球税率。我们将使用 None 函数参数：

In [26]:
# Let's now define a global tax rate. We'll use None for the function argument:
# the standard tax rate is 33%
+(tax_rate[None]==0.33)

# 一个函数可以在公式中调用：
# give me the net salary for all X
print((Z==salary[X]*(1-tax_rate[None])))

X   | Z                
----|------------------
foo | 46.89999999999999
bar | 73.69999999999999


在这种情况下，X受到salary[X]的约束，因此可以评估表达式。

一个函数也可以由一个子句定义。这是一个简单的例子：

In [27]:
# the net salary of X is Y if Y is the salary of X, reduced by the tax rate
net_salary[X] = salary[X]*(1-tax_rate[None])

# give me all the X and Y so that Y is the net salary of X
print(net_salary[X]==Y)

X   | Y                
----|------------------
foo | 46.89999999999999
bar | 73.69999999999999


In [28]:
# give me the net salary of foo
print(net_salary['foo']==Y)
print(net_salary[X]<50)

Y                
-----------------
46.89999999999999
X  
---
foo


现在让我们来定义一个累进税制：默认税率是33％，但是100％以上的工资是50％：

In [29]:
# Let's now define a progressive tax system: the tax rate is 33 % by default, but 50% for salaries above 100.
(tax_rate_for_salary_above[X] == 0.33) <= (0 <= X)
(tax_rate_for_salary_above[X] == 0.50) <= (100 <= X)
print(tax_rate_for_salary_above[70]==Y)
print(tax_rate_for_salary_above[150]==Y)

Y   
----
0.33
Y  
---
0.5


## 这里第一次出现了“推理”
"<="是上述陈述中的重要标志：它被读作'if'。【可以用来定义“推出”的规则】

首先给出函数的最一般定义。当搜索可能的答案时，pyDatalog从最后定义的规则开始，即更具体的规则，只要找到该函数的有效答案就立即停止。所以，尽管这两条规则似乎都适用于150的薪水，但实际上我们是按照第二条规则得到了50％的税率。

接下来让我们重新定义净工资。在此之前，我们要删除原始定义：

In [31]:
# new definition
net_salary[X] = salary[X]*(1-tax_rate_for_salary_above[salary[X]])
# give me all X and Y so that Y is the net salary of X
print(net_salary[X]==Y)
# Please note that we used f[X]=<expr> above, as a shorter notation for (f[X]==Y) <= (Y==expr)

# This short notation, together with the fact that functions can be defined in any order,
# makes writing a pyDatalog program as easy as creating a spreadsheet.

X   | Y                
----|------------------
foo | 46.89999999999999
bar | 55.0             


请注意，我们在上面使用的f[X]=，是(f[X]==Y) <= (Y==expr)的简写。

这个简短的表示法以及可以按任意顺序定义函数的事实，使得编写pyDatalog程序像创建电子表格一样简单。

为了说明这一点，看看这个不能更清晰的Factorial的定义：

In [32]:
# To illustrate the point, this definition of Factorial cannot be any clearer !
pyDatalog.create_terms('N, factorial')
factorial[N] = N*factorial[N-1]
factorial[1] = 1

print(factorial[3]==N)

N
-
6
